### Training a simple convolutional neural network on the CIFAR-10 dataset

#### Import packages
Here we are using [Keras](https://github.com/keras-team/keras) with [TensorFlow](https://www.tensorflow.org/) backend to develop a **convolutional neural network (CNN)** to training the classification of images from the [CIFAR-10 dataset] (https://www.toronto.edu/~kriz/cifar.html). 


#### Introduction
The dataset consists of airplanes, dogs, cats, and other objects. I'll preprocess the images, then train a convolutional neural network on all the samples. The images need to be normalized and the labels need to be one-hot encoded. 

In [122]:
from __future__ import print_function
import keras
from keras import callbacks
from keras.datasets import cifar10
from keras.utils import print_summary, to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation
from keras.optimizers import SGD
from keras import backend as K
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use('ggplot')

import time
start = time.time()

#### Initializing Hyperparameters
We will feed the CNN with the images as batches of 64 images in 100 epochs. The model will output the probabilities of 10 different categories (num_classes)

In [123]:
batch_size = 64
num_classes = 10
epochs = 100
model_name = 'keras_cifar10_model'
save_dir = '/model' + model_name

We explicitly define our MNIST image dimensions which will be used later to reshape our date.

In [124]:
img_rows, img_cols = 32, 32

#### Data Downloading
[CIFAR-10 dataset for Python](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz) will be downloaded from keras datasets, i.e. cifar10.

In [125]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#### Data preprocessing 

In [126]:
len(x_train)

50000

The labels in the dataset are converted into categorical matrix structure from 1-dim numpy array structure using <b>keras.utils.to_categorical(y, num_classes=None, dtype='float')</b> <br />
(source: https://keras.io/utils/)

In [127]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Comment out the two lines below to remove the normalization
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


#### Building the CNN Model
I will build the most common neural network model architecture in the format of [CONV]-[MaxPooling]-...-[CONV]-[MaxPooling]-[Dense]

In [128]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(80))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [129]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_26 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_27 (Activation)   (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 8, 8, 64)          0         
__________

#### Optimization using Stochastic Gradient Descent algorithm
SGD is used to optimize the weights on the backpropagation. Momentum parameter is set as 0.9 and other parameters are left as default. <br />
(Ref: https://leon.bottou.org/publications/pdf/compstat-2010.pdf, https://keras.io/optimizers/)

In [130]:
# lr=learning_rate, momentum=Parameter that accelerates SGD in the relevant direction and dampens oscillations
# decay=learning rate decay, nesterov= Whether to apply Nesterov momentum.
opt = SGD(lr=0.01, momentum=0.9, decay=0, nesterov=False) 

In [131]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

#### Training the model
I split the training dataset (50000 images) into training (40000 images) and validation (10000 images) datasets to measure the validation accuracy of our model. Thus, the neural network model will continue the training by evaluating the images that are not been during the training after each epoch.

In [132]:
import time
start = time.time()

In [133]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,
          shuffle=True)

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 [==============================] - 65s 2ms/step - loss: 2.0050 - acc: 0.2549 - val_loss: 1.6429 - val_acc: 0.4002
Epoch 2/100
40000/40000 [==============================] - 65s 2ms/step - loss: 1.6184 - acc: 0.4019 - val_loss: 1.4370 - val_acc: 0.4691
Epoch 3/100
40000/40000 [==============================] - 64s 2ms/step - loss: 1.4666 - acc: 0.4654 - val_loss: 1.3169 - val_acc: 0.5249
Epoch 4/100
40000/40000 [==============================] - 72s 2ms/step - loss: 1.3709 - acc: 0.4971 - val_loss: 1.2686 - val_acc: 0.5401
Epoch 5/100
40000/40000 [==============================] - 64s 2ms/step - loss: 1.2861 - acc: 0.5372 - val_loss: 1.1296 - val_acc: 0.5971
Epoch 6/100
40000/40000 [==============================] - 66s 2ms/step - loss: 1.2257 - acc: 0.5598 - val_loss: 1.1114 - val_acc: 0.6027
Epoch 7/100
40000/40000 [==============================] - 68s 2ms/step - loss: 1.1695 - acc: 0.5822 - val_loss: 1.0896 - 

40000/40000 [==============================] - 65s 2ms/step - loss: 0.7096 - acc: 0.7489 - val_loss: 0.6835 - val_acc: 0.7658
Epoch 60/100
40000/40000 [==============================] - 78s 2ms/step - loss: 0.7163 - acc: 0.7486 - val_loss: 0.7127 - val_acc: 0.7653
Epoch 61/100
40000/40000 [==============================] - 64s 2ms/step - loss: 0.7033 - acc: 0.7505 - val_loss: 0.7362 - val_acc: 0.7501
Epoch 62/100
40000/40000 [==============================] - 60s 1ms/step - loss: 0.7090 - acc: 0.7500 - val_loss: 0.6366 - val_acc: 0.7824
Epoch 63/100
40000/40000 [==============================] - 67s 2ms/step - loss: 0.7017 - acc: 0.7541 - val_loss: 0.6567 - val_acc: 0.7743
Epoch 64/100
40000/40000 [==============================] - 68s 2ms/step - loss: 0.7103 - acc: 0.7488 - val_loss: 0.6536 - val_acc: 0.7751
Epoch 65/100
40000/40000 [==============================] - 65s 2ms/step - loss: 0.7035 - acc: 0.7531 - val_loss: 0.6548 - val_acc: 0.7748
Epoch 66/100
40000/40000 [==============

In [134]:
end = time.time()
print ('Time taken to train model with epoch size {}: {} seconds'.format(epochs, end-start))

Time taken to train model with epoch size 100: 42825.29988884926 seconds


In [135]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 10s 995us/step
Test loss: 0.6422443160057068
Test accuracy: 0.7798


#### Results:
20 epochs - Accuracy: 68.70%, Loss: 0.89, Time Taken: 1183.97 seconds <br />
50 epochs - Accuracy: 72.86%, Loss: 0.73, Time Taken: 26853.56 seconds <br />
100 epochs - Accuracy: 77.98%, Loss: 0.64, Time Taken: 42825.30 seconds<br />
<p>
From the results, it is obvious that higher epochs leads to better performance.

#### Verifying the wrongly classified data

In [136]:
output = model.predict_classes(x_test)
print(output)

[3 8 8 ... 5 1 7]


In [137]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
ind = np.where(np.equal(output, y_test)==0)

In [138]:
output.shape

(10000,)

In [139]:
y_test.shape

(10000, 1)

In [140]:
err_x = x_test[ind[0]]
err_y = output[ind[0]]
print (err_x.shape)

MemoryError: 

In [ ]:
examples_per_class = 3
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

for cls, cls_name in enumerate(classes):
  idxs = np.where(cls == err_y)
  idxs = np.random.choice(idxs[0], examples_per_class, replace=False)
  for i, idx in enumerate(idxs):
    plt.subplot(examples_per_class, len(classes), i * len(classes) + cls + 1)
    plt.imshow(err_x[idx].astype('uint8'), cmap = 'gray')
    plt.axis('off')
    if i == 0:
      plt.title(cls_name)

#### Future Works and Improvements
<b> Changing Optimizer</b> - We can use other algorithm to optimize the weights and observe the results. <br />
<b> Changing Learning Rate</b> - We can decrease/increase the learning rate and observe the results.